### 2

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")


import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.03
MAGIC_NUMBER = 10001
LOG_FILE = "trade_log.txt"

# === MT5 CONNECT ===
def connect_mt5():
    if not mt5.initialize():
        logging.error("❌ MT5 initialization failed!")
        quit()
    logging.info("✅ MT5 initialized successfully.")

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(df, period=14):
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = df['true_range'].rolling(period).mean()
    return df

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    atr = df.iloc[-1]['ATR']
    support = df['low'].rolling(10).min().iloc[-1]
    resistance = df['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 5)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 5)

    return round(stop_loss, 2), round(take_profit, 2)

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if df.iloc[-1]['high'] > df.iloc[-2]['high']:
        signal = "BUY"
        entry_price = df.iloc[-1]['close']
    elif df.iloc[-1]['low'] < df.iloc[-2]['low']:
        signal = "SELL"
        entry_price = df.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        


# === RUN THE BOT ===
def run_bot():
    connect_mt5()
    while True:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")

        time.sleep(300)

run_bot()



### Tony

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MAGIC_NUMBER = 10002
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 200  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=,
        password="",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === CHECK PROFIT AND CLOSE ALL TRADES ===
def check_profit_and_exit():
    global INITIAL_BALANCE
    account_info = mt5.account_info()
    if account_info is None:
        logging.error("❌ Could not fetch account info.")
        return

    net_profit = account_info.balance - INITIAL_BALANCE
    logging.info(f"💹 Current Net Profit: ${net_profit:.2f}")

    if net_profit >= PROFIT_TARGET:
        logging.info("🎯 Profit target reached. Closing all positions...")
        positions = mt5.positions_get()
        if positions:
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": pos.symbol,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "position": pos.ticket,
                    "price": mt5.symbol_info_tick(pos.symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(pos.symbol).ask,
                    "deviation": 20,
                    "magic": MAGIC_NUMBER,
                    "comment": "Close on profit target",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    logging.info(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    logging.warning(f"⚠️ Failed to close position {pos.ticket}: {result.retcode}")
        logging.info("🛑 Stopping bot after reaching profit target.")
        mt5.shutdown()
        exit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(df, period=14):
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = df['true_range'].rolling(period).mean()
    return df

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    atr = df.iloc[-1]['ATR']
    support = df['low'].rolling(10).min().iloc[-1]
    resistance = df['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return round(stop_loss, 2), round(take_profit, 2)

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if df.iloc[-1]['high'] > df.iloc[-2]['high']:
        signal = "BUY"
        entry_price = df.iloc[-1]['close']
    elif df.iloc[-1]['low'] < df.iloc[-2]['low']:
        signal = "SELL"
        entry_price = df.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def equity_check():
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return
    
    inital_balance = account_info.balance
    target_equity = inital_balance + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is not None:
            equity = account_info.equity

            if equity >= target_equity:
                print("🚀 Equity target reached! Closing all open positions...")
                positions = mt5.positions_get()
                if positions:
                    for pos in positions:
                        symbol = pos.symbol
                        volume = pos.volume
                        ticket = pos.ticket
                        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                        price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                        close_request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": ticket,
                            "symbol": symbol,
                            "volume": volume,
                            "type": order_type,
                            "price": price,
                            "deviation": 10,
                            "magic": 123456,
                            "comment": "Auto close on equity profit"
                        }

                        result = mt5.order_send(close_request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"✔️ Closed position {ticket}")
                        else:
                            print(f"❌ Failed to close position {ticket}, retcode: {result.retcode}")
        else:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")

        time.sleep(60)

# === RUN THE BOT ===
def run_bot():
    connect_mt5()
    target_profit = 200  # Target profit in USD
    initial_equity = mt5.account_info().equity  # Get initial equity of the account

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check, daemon=True).start()
    
    while True:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Check if equity has reached the target profit
        current_equity = mt5.account_info().equity
        if current_equity >= initial_equity + target_profit:
            print(f"✅ Equity reached ${target_profit} profit. Closing all positions...")
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": pos.ticket,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "price": mt5.symbol_info_tick(SYMBOL).ask if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).bid,
                    "deviation": 10,
                    "magic": 123456,
                    "comment": "Closing positions after reaching profit target",
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    print(f"❌ Failed to close position {pos.ticket}: {result.retcode}")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")

        time.sleep(300)


# Start the trading bot
run_bot()

✅ MT5 initialized and logged in successfully.
✅ Initial Balance: 1717.83
🎯 Monitoring for equity to reach $1917.83...
📢 Trade Signal: BUY | Entry: 3240.947 | SL: 3233.55 | TP: 3255.74
✅ BUY Order Placed: 3240.947
✅ SL/TP Updated: SL=3233.55, TP=3255.74
✅ BUY SL updated for position 1900485662 → 3234.59
⚠️ Trade skipped: Already in a BUY position.
⏳ Waiting for new opposite signal...
✅ BUY SL updated for position 1900485662 → 3235.91
⚠️ Trade skipped: Already in a BUY position.
⏳ Waiting for new opposite signal...
✅ BUY SL updated for position 1900485662 → 3237.22
⏳ No trade opportunity found.
✅ BUY SL updated for position 1900485662 → 3238.12
⏳ No trade opportunity found.
✅ BUY SL updated for position 1900485662 → 3240.52
⚠️ Trade skipped: Already in a BUY position.
⏳ Waiting for new opposite signal...
✅ BUY SL updated for position 1900485662 → 3243.92
⏳ No trade opportunity found.
📢 Trade Signal: BUY | Entry: 3241.141 | SL: 3234.88 | TP: 3253.66
✅ BUY Order Placed: 3241.141
✅ SL/TP Up